In [1]:
import ee
import datetime
import os
import itertools
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

##### Define_Basic_Parameters

In [17]:
# set the num of different sample points
num_invarient = 10
num_each_year = 5



# define the year range
year_start = [i for i in range(1990,2018,3)]
year_end   = [i for i in range(1992,2020,3)]

year_name = [f'{i[0]}_{i[1]}' for i in zip(year_start,year_end)]

In [5]:
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [7]:
year_name

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019']

##### Prepare needed images

In [9]:
# import Landsat img
Landsat_img = [ee.Image(f"users/wang8052664/Cloud_Free_Img/Landsat_cloud_free_{year}")\
                 .clip(North_China_Plain)  for year in year_name]

# import Fourier img   
Fourier_img = [ee.Image(f"users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_{year}_harmonic_3")\
                     .clip(North_China_Plain)  for year in year_name[:5]] + \
                  [ee.Image(f"users/wang8052664/North_China_Plain/Fourier_imgs/Fourier_img_{year}_harmonic_3")\
                     .clip(North_China_Plain)  for year in year_name[5:]] 

# Import the Index-mean Img
NDVI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDVI").rename('Mean_NDVI')  
            for year in year_name ]
NDBI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDBI").rename('Mean_NDBI')  
            for year in year_name ]
EVI_img  = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_EVI").rename('Mean_EVI')  
            for year in year_name ]

# stack Landsat_Fourier_Nomalized together
Stack_img = [ee.Image(img) for img in zip(Landsat_img,Fourier_img,NDVI_img,NDBI_img,EVI_img)]

##### Img value extraction and export

In [108]:
# Sample points serves as INVARIENT points
Invarient_non_built = [ee.FeatureCollection('''users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Verified_pt_2017_2019''')\
                        .filterMetadata('Built','equals',0)] *  num_invarient

Invarient_built     = [ee.FeatureCollection('''users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Village_Point_visual_checked_1990_1992''')]\
                         *  num_invarient



# Sample points serves as EACH_YEAR points
# NOTE: Each_year_visual_check has [9K non-built] and [`1K built]
Each_year_visual_check = [ee.FeatureCollection(f"users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Verified_pt_{year}")
                          for year in year_name[-num_each_year:] ]

# Each_year_built serves as checked-built points, 
# which was create by polygon_city_rurul to raster
# based on base map of 1990_1992
Each_year_built        = [ee.FeatureCollection("users/Jinzhu_Deakin/North_China_Plain/Zone_sample_point_1990_1992/Zone_Sample")]\
                         *  num_invarient


In [109]:
# define the gee-asset path for exporting
path = 'users/Jinzhu_Deakin/North_China_Plain/Sample_with_Landsat_Fourier_Normalized'


for year,img,sample in list(zip(year_name,Stack_img,Each_year_built)):
    
    name = 'Each_year_built'
    
    Ext_sample = img.sampleRegions(collection=sample, 
                                    properties=['Built'], 
                                    scale=30, 
                                    geometries=True)
    
    # exporting
    task = ee.batch.Export.table.toAsset(   collection  = Ext_sample,
                                            description = f'{name}_{year}',
                                            assetId     = f'{path}/{name}_{year}')
    task.start()

    
    # print out the process
    print(f'{name}_{year}')

Each_year_built_1990_1992
Each_year_built_1993_1995
Each_year_built_1996_1998
Each_year_built_1999_2001
Each_year_built_2002_2004
Each_year_built_2005_2007
Each_year_built_2008_2010
Each_year_built_2011_2013
Each_year_built_2014_2016
Each_year_built_2017_2019
